# 🚢 Application de Tracking de Bateaux

Application moderne pour tracker des bateaux avec météo, géolocalisation, données (AIS, RF et imagerie satellite) et chatbot IA. Pour lancer, cliquer sur "Tout exécuter".


# 📦 Installation des dépendances Python



In [1]:
!pip install folium ipywidgets requests pandas plotly dash jupyter-dash geopy openai websockets nest_asyncio
!pip install -qU langchain langchain-openai langchain-community
!pip install copernicusmarine xarray netCDF4



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

# 🔑 Configuration des clés API (OpenWeather, OpenRouter, AISStream)


In [2]:
OPENWEATHER_API_KEY = ""
OPENROUTER_API_KEY = ""
AISSTREAM_API_KEY = ""


# URLs des APIs
OPENWEATHER_BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

# Configuration alternative pour le chatbot local (si OpenRouter ne fonctionne pas)
USE_LOCAL_CHATBOT = False  # Mettre à True pour utiliser un chatbot local simple


# 🗂️ Imports et configuration de base

In [3]:
import pandas as pd
import folium
from folium import plugins
import requests
import json
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration pour l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# 🏳️ Dictionnaire des drapeaux par pays

In [4]:
FLAG_EMOJIS = {
    # Pays des bateaux de la flotte
    'France': '🇫🇷',
    'Royaume-Uni': '🇬🇧',
    'États-Unis': '🇺🇸',
    'Japon': '🇯🇵',
    'Italie': '🇮🇹',
    'Suède': '🇸🇪',
    'Finlande': '🇫🇮',
    'Bahamas': '🇧🇸',
    'Russie': '🇷🇺',
    'Pays-Bas': '🇳🇱',
    'Hong Kong': '🇭🇰',
    'Chine': '🇨🇳',
    'Islande': '🇮🇸',
    'Belgique': '🇧🇪',
    # Pays supplémentaires pour la géolocalisation
    'Espagne': '🇪🇸',
    'Allemagne': '🇩🇪',
    'Portugal': '🇵🇹',
    'Norvège': '🇳🇴',
    'Danemark': '🇩🇰',
    'Grèce': '🇬🇷',
    'Turquie': '🇹🇷',
    'Maroc': '🇲🇦',
    'Algérie': '🇩🇿',
    'Tunisie': '🇹🇳',
    'Égypte': '🇪🇬',
    'Canada': '🇨🇦',
    'Mexique': '🇲🇽',
    'Brésil': '🇧🇷',
    'Argentine': '🇦🇷',
    'Chili': '🇨🇱',
    'Australie': '🇦🇺',
    'Nouvelle-Zélande': '🇳🇿',
    'Afrique du Sud': '🇿🇦',
    'Inde': '🇮🇳',
    'Corée du Sud': '🇰🇷',
    'Thaïlande': '🇹🇭',
    'Indonésie': '🇮🇩',
    'Philippines': '🇵🇭',
    'Singapour': '🇸🇬',
    'Malaisie': '🇲🇾',
    'Vietnam': '🇻🇳',
    'Israël': '🇮🇱',
    'Émirats arabes unis': '🇦🇪',
    'Arabie saoudite': '🇸🇦',
    # Noms anglais pour compatibilité avec l'API de géolocalisation
    'United Kingdom': '🇬🇧',
    'United States': '🇺🇸',
    'Japan': '🇯🇵',
    'Italy': '🇮🇹',
    'Sweden': '🇸🇪',
    'Finland': '🇫🇮',
    'Russia': '🇷🇺',
    'Netherlands': '🇳🇱',
    'China': '🇨🇳',
    'Iceland': '🇮🇸',
    'Belgium': '🇧🇪',
    'Spain': '🇪🇸',
    'Germany': '🇩🇪',
    'Norway': '🇳🇴',
    'Denmark': '🇩🇰',
    'Greece': '🇬🇷',
    'Turkey': '🇹🇷',
    'Morocco': '🇲🇦',
    'Algeria': '🇩🇿',
    'Tunisia': '🇹🇳',
    'Egypt': '🇪🇬',
    'Mexico': '🇲🇽',
    'Brazil': '🇧🇷',
    'Argentina': '🇦🇷',
    'Chile': '🇨🇱',
    'Australia': '🇦🇺',
    'New Zealand': '🇳🇿',
    'South Africa': '🇿🇦',
    'India': '🇮🇳',
    'South Korea': '🇰🇷',
    'Thailand': '🇹🇭',
    'Indonesia': '🇮🇩',
    'Philippines': '🇵🇭',
    'Singapore': '🇸🇬',
    'Malaysia': '🇲🇾',
    'Vietnam': '🇻🇳',
    'Israel': '🇮🇱',
    'United Arab Emirates': '🇦🇪',
    'Saudi Arabia': '🇸🇦',
    # Cas spéciaux
    'En mer': '🌊',
    'Inconnu': '🏳️',
    'Erreur': '❌'
}

def get_flag_emoji(country):
    """Retourne l'emoji du drapeau pour un pays donné"""
    return FLAG_EMOJIS.get(country, '🏳️')  # Drapeau blanc par défaut

# 🌤️ Fonctions API météo OpenWeather

In [5]:
def get_weather_data(lat, lon, api_key):
    """
    Récupère les données météo pour une position donnée
    """
    try:
        url = f"{OPENWEATHER_BASE_URL}?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=fr"
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Erreur API: {response.status_code}"}
    except Exception as e:
        return {"error": f"Erreur de connexion: {str(e)}"}

def format_weather_info(weather_data):
    """
    Formate les informations météo pour l'affichage
    """
    if "error" in weather_data:
        return f"❌ {weather_data['error']}"

    try:
        temp = weather_data['main']['temp']
        feels_like = weather_data['main']['feels_like']
        humidity = weather_data['main']['humidity']
        pressure = weather_data['main']['pressure']
        description = weather_data['weather'][0]['description'].title()
        wind_speed = weather_data['wind']['speed']
        wind_deg = weather_data['wind'].get('deg', 0)

        weather_info = f"""
        🌡️ **Température**: {temp}°C (ressenti {feels_like}°C)
        🌤️ **Conditions**: {description}
        💨 **Vent**: {wind_speed} m/s, {wind_deg}°
        💧 **Humidité**: {humidity}%
        📊 **Pression**: {pressure} hPa
        """

        return weather_info.strip()
    except KeyError as e:
        return f"❌ Données météo incomplètes: {str(e)}"

print("✅ Fonctions météo configurées!")


✅ Fonctions météo configurées!


# 🌍 Fonctions de géolocalisation et zones maritimes

In [6]:
# 🌍 Fonctions de géolocalisation avec zones maritimes précises (eaux territoriales)

import math

def get_location_info(lat, lon):
    """
    Récupère les informations de localisation avec zones maritimes précises
    """
    try:
        geolocator = Nominatim(user_agent="boat_tracker_app")
        location = geolocator.reverse(
            f"{lat}, {lon}",
            timeout=10,
            language='fr'
        )

        if location:
            # Cas 1: Position terrestre/portuaire détectée par OSM
            address = location.raw['address']
            country = translate_to_french(address.get('country', 'Inconnu'))
            city = translate_to_french(
                address.get('city') or
                address.get('town') or
                address.get('village') or
                'Zone portuaire'
            )
            state = translate_to_french(address.get('state', ''))
            water_body = get_water_body(lat, lon)

            return {
                'country': country,
                'city': city,
                'state': state,
                'water_body': water_body,
                'full_address': translate_to_french(location.address),
                'maritime_status': 'Port/Côte'
            }
        else:
            # Cas 2: Position maritime - analyser les zones
            maritime_zone = get_precise_maritime_zone(lat, lon)

            return {
                'country': maritime_zone['country'],
                'city': maritime_zone['zone_type'],
                'state': '',
                'water_body': maritime_zone['water_body'],
                'full_address': maritime_zone['description'],
                'maritime_status': maritime_zone['status']
            }

    except GeocoderTimedOut:
        return {
            'country': 'Timeout',
            'city': 'Erreur de géolocalisation',
            'state': '',
            'water_body': 'Inconnu',
            'full_address': 'Erreur de connexion',
            'maritime_status': 'Erreur'
        }
    except Exception as e:
        return {
            'country': 'Erreur',
            'city': str(e),
            'state': '',
            'water_body': 'Inconnu',
            'full_address': f"Erreur: {str(e)}",
            'maritime_status': 'Erreur'
        }

def get_precise_maritime_zone(lat, lon):
    """
    Détermine la zone maritime précise selon le droit international
    """
    # 1. Déterminer la masse d'eau
    water_body = get_water_body(lat, lon)

    # 2. Trouver le pays côtier le plus proche
    closest_country_info = find_closest_coastal_country(lat, lon)
    closest_country = closest_country_info['country']
    distance_nm = closest_country_info['distance_nm']

    # 3. Déterminer la zone maritime selon le droit international
    if distance_nm <= 12:  # Eaux territoriales (12 milles nautiques)
        zone_type = "Eaux territoriales"
        country = closest_country
        status = f"Territorial ({distance_nm:.1f} nm de la côte)"
        description = f"Eaux territoriales {closest_country} - {distance_nm:.1f} nm de la côte"

    elif distance_nm <= 24:  # Zone contiguë (24 nm)
        zone_type = "Zone contiguë"
        country = f"Zone contiguë {closest_country}"
        status = f"Contiguë ({distance_nm:.1f} nm de la côte)"
        description = f"Zone contiguë {closest_country} - {distance_nm:.1f} nm de la côte"

    elif distance_nm <= 200:  # Zone économique exclusive (200 nm)
        zone_type = "ZEE"
        country = f"ZEE {closest_country}"
        status = f"ZEE ({distance_nm:.1f} nm de la côte)"
        description = f"Zone Économique Exclusive {closest_country} - {distance_nm:.1f} nm de la côte"

    else:  # Eaux internationales (> 200 nm)
        zone_type = "Eaux internationales"
        country = "Eaux internationales"
        status = f"International ({distance_nm:.1f} nm de la côte)"
        description = f"Eaux internationales - {distance_nm:.1f} nm de {closest_country}"

    return {
        'country': country,
        'zone_type': zone_type,
        'water_body': water_body,
        'status': status,
        'description': description,
        'distance_nm': distance_nm,
        'closest_country': closest_country
    }

def find_closest_coastal_country(lat, lon):
    """
    Trouve le pays côtier le plus proche et calcule la distance
    """
    # Points côtiers de référence pour les principaux pays méditerranéens
    coastal_references = {
        'France': [
            (43.2965, 5.3698),   # Marseille
            (43.7102, 7.2620),   # Nice
            (42.6976, 2.8954),   # Perpignan/Port-Vendres
            (43.1258, 6.1266),   # Toulon
            (43.5528, 3.8330),   # Montpellier/Sète
        ],
        'Espagne': [
            (41.3851, 2.1734),   # Barcelone
            (36.7201, -4.4203),  # Malaga
            (37.3886, -5.9823),  # Séville
            (39.4699, 2.7581),   # Palma de Majorque
            (36.1408, -5.3536),  # Gibraltar
        ],
        'Italie': [
            (40.8518, 14.2681),  # Naples
            (41.9028, 12.4964),  # Rome/Civitavecchia
            (44.4056, 8.9463),   # Gênes
            (45.4408, 12.3155),  # Venise
            (38.1157, 13.3615),  # Palerme
            (40.6401, 17.9443),  # Brindisi
        ],
        'Grèce': [
            (37.9755, 23.7348),  # Athènes/Pirée
            (40.6401, 22.9444),  # Thessalonique
            (35.5138, 24.0180),  # Crète/Héraklion
            (39.6243, 19.9217),  # Corfou
            (36.4414, 25.3656),  # Santorin
        ],
        'Turquie': [
            (41.0082, 28.9784),  # Istanbul
            (36.8969, 30.7133),  # Antalya
            (38.4192, 27.1287),  # Izmir
            (36.2012, 36.1610),  # Antakya
        ],
        'Chypre': [
            (35.1856, 33.3823),  # Nicosie/Limassol
            (35.0408, 33.9533),  # Larnaca
        ],
        'Croatie': [
            (45.8150, 15.9819),  # Zagreb/Rijeka
            (42.6507, 18.0944),  # Dubrovnik
            (43.5081, 16.4402),  # Split
        ],
        'Algérie': [
            (36.7538, 3.0588),   # Alger
            (35.6976, -0.6337),  # Oran
        ],
        'Tunisie': [
            (36.8065, 10.1815),  # Tunis
            (34.7406, 10.7603),  # Sfax
        ],
        'Maroc': [
            (35.7595, -5.8340),  # Tanger
            (33.5731, -7.5898),  # Casablanca
        ],
        'Égypte': [
            (31.2001, 29.9187),  # Alexandrie
            (27.2579, 33.8116),  # Hurghada
        ],
        'Malte': [
            (35.8997, 14.5146),  # La Valette
        ],
        'Monaco': [
            (43.7384, 7.4246),   # Monaco
        ]
    }

    min_distance = float('inf')
    closest_country = 'Inconnu'

    for country, points in coastal_references.items():
        for point_lat, point_lon in points:
            distance = haversine_nm(lat, lon, point_lat, point_lon)
            if distance < min_distance:
                min_distance = distance
                closest_country = country

    return {
        'country': closest_country,
        'distance_nm': min_distance
    }

def haversine_nm(lat1, lon1, lat2, lon2):
    """
    Calcule la distance en milles nautiques entre deux points GPS
    """
    R = 3440.065  # Rayon de la Terre en milles nautiques

    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    return R * c

def get_water_body(lat, lon):
    """
    Détermine la masse d'eau avec plus de précision
    """
    # Méditerranée (étendue)
    if 30 <= lat <= 46 and -6 <= lon <= 42:
        # Sous-zones méditerranéennes
        if 40 <= lat <= 46 and 3 <= lon <= 20:
            return "Mer Méditerranée occidentale"
        elif 32 <= lat <= 42 and 15 <= lon <= 30:
            return "Mer Méditerranée orientale"
        elif 35 <= lat <= 42 and 12 <= lon <= 20:
            return "Mer Adriatique"
        elif 35 <= lat <= 40 and 19 <= lon <= 30:
            return "Mer Ionienne"
        elif 35 <= lat <= 42 and 22 <= lon <= 30:
            return "Mer Égée"
        else:
            return "Mer Méditerranée"

    # Mer Noire
    elif 40 <= lat <= 48 and 27 <= lon <= 42:
        return "Mer Noire"

    # Mer du Nord / Baltique
    elif 53 <= lat <= 70 and -5 <= lon <= 30:
        if lat >= 60:
            return "Mer de Norvège"
        elif 54 <= lat <= 66 and 10 <= lon <= 30:
            return "Mer Baltique"
        else:
            return "Mer du Nord"

    # Océan Atlantique
    elif -60 <= lat <= 70 and -80 <= lon <= 0:
        if 35 <= lat <= 70:
            return "Océan Atlantique Nord"
        elif 0 <= lat <= 35:
            return "Océan Atlantique tropical"
        else:
            return "Océan Atlantique Sud"

    # Océan Pacifique
    elif -60 <= lat <= 60 and 120 <= lon <= 180:
        return "Océan Pacifique"

    # Océan Indien
    elif -60 <= lat <= 30 and 20 <= lon <= 120:
        return "Océan Indien"

    # Océan Arctique
    elif lat >= 70:
        return "Océan Arctique"

    # Océan Antarctique
    elif lat <= -60:
        return "Océan Antarctique"

    else:
        return "Zone maritime"

def translate_to_french(text):
    """
    Traduit les noms de lieux vers le français avec dictionnaire de correspondances
    """
    if not text or text in ['', 'Inconnu', 'En mer', 'Erreur']:
        return text

    # Dictionnaire de traductions (identique à avant mais étendu)
    translations = {
        # Grèce
        'Ελλάς': 'Grèce', 'Ελλάδα': 'Grèce', 'Greece': 'Grèce',
        'Κυψέλη': 'Kypseli', 'Αθήνα': 'Athènes', 'Θεσσαλονίκη': 'Thessalonique',
        'Πειραιάς': 'Le Pirée', 'Πάτρα': 'Patras', 'Κρήτη': 'Crète',
        'Σαντορίνη': 'Santorin', 'Μύκονος': 'Mykonos', 'Ρόδος': 'Rhodes',
        'Κέρκυρα': 'Corfou', 'Επισκοπή': 'Episkopi', 'Επίσκοπι': 'Episkopi',

        # Chypre
        'Cyprus': 'Chypre', 'Κύπρος': 'Chypre', 'Kıbrıs': 'Chypre',
        'Episkopi': 'Episkopi', 'Επισκοπή': 'Episkopi',

        # Espagne
        'España': 'Espagne', 'Barcelona': 'Barcelone', 'Sevilla': 'Séville',
        'Málaga': 'Malaga', 'Valencia': 'Valence', 'Zaragoza': 'Saragosse',

        # Italie
        'Italia': 'Italie', 'Roma': 'Rome', 'Milano': 'Milan',
        'Napoli': 'Naples', 'Venezia': 'Venise', 'Firenze': 'Florence',
        'Genova': 'Gênes', 'Palermo': 'Palerme',

        # Turquie
        'Türkiye': 'Turquie', 'Turkey': 'Turquie', 'İstanbul': 'Istanbul',
        'İzmir': 'Izmir', 'Antalya': 'Antalya',

        # Pays arabes
        'المغرب': 'Maroc', 'Morocco': 'Maroc', 'مصر': 'Égypte', 'Egypt': 'Égypte',
        'الجزائر': 'Algérie', 'Algeria': 'Algérie', 'تونس': 'Tunisie', 'Tunisia': 'Tunisie',

        # Autres
        'Malta': 'Malte', 'Croatia': 'Croatie', 'Hrvatska': 'Croatie',
        'Albania': 'Albanie', 'Shqipëria': 'Albanie',
        'United States': 'États-Unis', 'United Kingdom': 'Royaume-Uni',
        'Germany': 'Allemagne', 'Netherlands': 'Pays-Bas',
    }

    # Recherche exacte puis partielle
    if text in translations:
        return translations[text]

    result = text
    for original, french in translations.items():
        if original in result:
            result = result.replace(original, french)

    return result

def get_country_flag_smart(country_name):
    """Détecte intelligemment le drapeau d'un pays même avec des noms multilingues"""
    if not country_name or country_name in ['Inconnu', 'En mer', 'Erreur']:
        return get_flag_emoji(country_name)

    country_lower = country_name.lower()

    # Mappings étendus pour les zones maritimes
    country_mappings = {
        'france': '🇫🇷', 'grèce': '🇬🇷', 'greece': '🇬🇷', 'ελλάς': '🇬🇷', 'ελλάδα': '🇬🇷',
        'espagne': '🇪🇸', 'spain': '🇪🇸', 'españa': '🇪🇸',
        'italie': '🇮🇹', 'italy': '🇮🇹', 'italia': '🇮🇹',
        'turquie': '🇹🇷', 'turkey': '🇹🇷', 'türkiye': '🇹🇷',
        'chypre': '🇨🇾', 'cyprus': '🇨🇾', 'κύπρος': '🇨🇾',
        'croatie': '🇭🇷', 'croatia': '🇭🇷', 'hrvatska': '🇭🇷',
        'algérie': '🇩🇿', 'algeria': '🇩🇿', 'الجزائر': '🇩🇿',
        'tunisie': '🇹🇳', 'tunisia': '🇹🇳', 'تونس': '🇹🇳',
        'maroc': '🇲🇦', 'morocco': '🇲🇦', 'المغرب': '🇲🇦',
        'égypte': '🇪🇬', 'egypt': '🇪🇬', 'مصر': '🇪🇬',
        'malte': '🇲🇹', 'malta': '🇲🇹',
        'albanie': '🇦🇱', 'albania': '🇦🇱',
        'monténégro': '🇲🇪', 'montenegro': '🇲🇪',
        'eaux internationales': '🌊', 'international': '🌊',
        'zee': '🌊', 'zone contiguë': '🌊', 'eaux territoriales': '🌊',
    }

    # Recherche avec gestion des zones maritimes
    for key, flag in country_mappings.items():
        if key in country_lower:
            return flag

    return get_flag_emoji(country_name)

def format_location_info(location_data):
    """Formate les informations de localisation avec zones maritimes précises"""
    country = location_data['country']
    flag = get_country_flag_smart(country)

    # Affichage adapté selon le type de zone
    if location_data.get('maritime_status') == 'Port/Côte':
        location_info = f"""
        {flag} **Pays**: {country}
        🏙️ **Ville**: {location_data['city']}
        🌊 **Zone maritime**: {location_data['water_body']}
        📍 **Adresse complète**: {location_data['full_address']}
        """
    else:
        # Zone maritime
        status = location_data.get('maritime_status', '')
        location_info = f"""
        {flag} **Zone**: {country}
        🌊 **Statut**: {location_data['city']} ({status})
        🌊 **Masse d'eau**: {location_data['water_body']}
        📍 **Position**: {location_data['full_address']}
        """

    return location_info.strip()

print("✅ Fonctions de géolocalisation configurées avec zones maritimes précises!")

✅ Fonctions de géolocalisation configurées avec zones maritimes précises!


# 🤖 Fonctions chatbot (OpenRouter + fallback local)

In [7]:
def query_openrouter(question, context_data, api_key):
    """
    Interroge l'API OpenRouter avec les données contextuelles
    """
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com",
            "X-Title": "Boat Tracker App"
        }

        # Préparation du contexte
        context = f"""
        Tu es un assistant spécialisé dans le tracking maritime.
        Voici les données des bateaux disponibles:
        {context_data}

        Réponds de manière précise et utile aux questions sur ces bateaux.
        """

        payload = {
            "model": "deepseek/deepseek-chat-v3.1:free",
            "messages": [
                {"role": "system", "content": context},
                {"role": "user", "content": question}
            ],
            "max_tokens": 500,
            "temperature": 0.7
        }

        response = requests.post(OPENROUTER_BASE_URL,
                               headers=headers,
                               json=payload,
                               timeout=30)

        if response.status_code == 200:
            result = response.json()
            return result['choices'][0]['message']['content']
        elif response.status_code == 402:
            return "❌ Erreur 402: Crédit insuffisant sur votre compte OpenRouter. Vérifiez votre solde sur https://openrouter.ai/credits"
        elif response.status_code == 401:
            return "❌ Erreur 401: Clé API OpenRouter invalide. Vérifiez votre clé sur https://openrouter.ai/keys"
        elif response.status_code == 429:
            return "❌ Erreur 429: Limite de taux dépassée. Attendez quelques minutes avant de réessayer."
        else:
            return f"❌ Erreur API OpenRouter: {response.status_code} - {response.text}"

    except Exception as e:
        return f"❌ Erreur de connexion au chatbot: {str(e)}"

def local_chatbot(question, context_data):
    """
    Chatbot local simple basé sur des règles (alternative à OpenRouter)
    """
    question_lower = question.lower()

    # Comptage des bateaux
    if any(word in question_lower for word in ['combien', 'nombre', 'total']):
        boat_count = context_data.count('⚪') + context_data.count('🔴')
        return f"Il y a {boat_count} bateaux dans la flotte actuellement trackés."

    # Types de bateaux
    if any(word in question_lower for word in ['type', 'catégorie', 'sorte']):
        types = ['Cargo', 'Research', 'Fishing', 'Container', 'Cruise', 'Tanker', 'Naval', 'Icebreaker', 'Tourism', 'Yacht']
        return f"Les types de bateaux dans la flotte sont : {', '.join(types)}"

    # Pays/pavillons
    if any(word in question_lower for word in ['pays', 'pavillon', 'nationalité', 'drapeau']):
        countries = ['France', 'United Kingdom', 'United States', 'Japan', 'Italy', 'Sweden', 'Finland', 'Bahamas', 'Russia', 'Netherlands', 'Hong Kong', 'China', 'Iceland', 'Belgium']
        return f"Les pavillons représentés sont : {', '.join(set(countries))}"

    # Bateau sélectionné
    if '🔴' in context_data:
        selected_line = [line for line in context_data.split('\n') if '🔴' in line]
        if selected_line:
            boat_info = selected_line[0].replace('🔴', '').strip()
            if any(word in question_lower for word in ['sélectionné', 'choisi', 'actuel', 'rouge']):
                return f"Le bateau actuellement sélectionné est : {boat_info}"

    # Localisation
    if any(word in question_lower for word in ['où', 'position', 'localisation', 'coordonnées']):
        return "Les bateaux sont répartis dans le monde entier. Sélectionnez un bateau pour voir sa position exacte et les informations de localisation."

    # Météo
    if any(word in question_lower for word in ['météo', 'temps', 'température', 'vent']):
        return "Les informations météo sont disponibles pour chaque bateau sélectionné. Choisissez un bateau pour voir les conditions météorologiques en temps réel."

    # Réponse par défaut
    return f"Je suis un assistant maritime local. Voici ce que je peux vous dire :\n\n{context_data}\n\nPosez-moi des questions sur le nombre de bateaux, leurs types, leurs pavillons, ou sélectionnez un bateau pour plus d'informations."

def create_context_from_boats(df, selected_boat=None):
    """
    Crée un contexte textuel à partir des données des bateaux
    """
    context = "Données des bateaux:\n"

    for _, boat in df.iterrows():
        marker = "🔴" if selected_boat and boat['boat_name'] == selected_boat else "⚪"
        context += f"{marker} {boat['boat_name']}: {boat['boat_type']}, {boat['flag_country']}, Position: ({boat['latitude']:.4f}, {boat['longitude']:.4f})\n"

    return context

print("✅ Fonctions chatbot configurées!")


✅ Fonctions chatbot configurées!


# 🗺️ Fonction création carte Folium avec bathymétrie EMODnet

In [8]:
def create_boat_map(df, selected_mmsi=None):
    """
    Crée une carte interactive centrée Méditerranée avec couche EMODnet,
    et met en évidence le navire sélectionné par MMSI.
    """
    center_lat, center_lon = 38.5, 15.0
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5, tiles='OpenStreetMap')

    folium.WmsTileLayer(
        url="https://ows.emodnet-bathymetry.eu/wms",
        name="Bathymétrie EMODnet",
        layers="emodnet:mean_atlas_land",
        fmt="image/png",
        transparent=True,
        overlay=True,
        control=True,
        attr="EMODnet Bathymetry"
    ).add_to(m)
    folium.LayerControl().add_to(m)

    # Marqueurs
    for _, boat in df.iterrows():
        lat, lon = float(boat["latitude"]), float(boat["longitude"])
        name = str(boat["boat_name"])
        mmsi = boat.get("mmsi")

        # ✅ CORRECTION: Comparaison robuste des MMSI
        is_selected = False
        if selected_mmsi is not None and mmsi is not None:
            try:
                # Comparaison en string ET en int pour être sûr
                is_selected = (str(mmsi) == str(selected_mmsi)) or (int(mmsi) == int(selected_mmsi))
            except (ValueError, TypeError):
                # Fallback: comparaison en string seulement
                is_selected = (str(mmsi) == str(selected_mmsi))

        color = 'red' if is_selected else 'blue'
        icon = 'star' if is_selected else 'ship'

        popup_text = f"""<b>{'🔴 ' if is_selected else ''}{name}</b><br>
                         MMSI: {mmsi}<br>
                         Type: {boat.get('boat_type', 'N/A')}<br>
                         Pavillon: {boat.get('flag_country', 'N/A')}<br>
                         Position: {lat:.4f}, {lon:.4f}"""

        folium.Marker(
            [lat, lon],
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"{name} — MMSI {mmsi}",
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(m)

    # Légende
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 220px; height: 100px;
                background-color: white; border:2px solid grey; z-index:9999;
                font-size:14px; padding: 10px">
    <h4>Légende</h4>
    <i class="fa fa-star" style="color:red"></i> Bateau sélectionné<br>
    <i class="fa fa-ship" style="color:blue"></i> Autres bateaux
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    return m


print("✅ Fonction de création de carte configurée (Méditerranée + EMODnet)!")


✅ Fonction de création de carte configurée (Méditerranée + EMODnet)!


# 💻 Widgets interface base + handler sélection navire (météo/localisation/carte)

In [9]:
# ⚠️ Attendu déjà défini ailleurs:
# - df_boats (fictif) pour fallback éventuel
# - get_weather_data(lat, lon, OPENWEATHER_API_KEY)
# - format_weather_info(...)
# - get_location_info(lat, lon), format_location_info(...)
# - create_boat_map(df, selected_mmsi=None)
# - vessels_state (dict live AIS) + vessels_state_to_df()

import ipywidgets as widgets
from IPython.display import display, clear_output

# État global
current_boat = None
current_weather = None
current_location = None

# Mémorisation du navire sélectionné (utilisé par les tools/agent)
last_selected_mmsi = None
last_selected_name = None

# Widgets
boat_selector = widgets.Dropdown(
    options=[('Sélectionnez un bateau', None)],  # ⚠️ sera rempli dynamiquement par refresh_ui_from_stream()
    value=None,
    description='Bateau:',
    style={'description_width': 'initial'}
)

weather_output = widgets.HTML(
    value="<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Sélectionnez un bateau pour voir la météo</div>"
)

location_output = widgets.HTML(
    value="<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Sélectionnez un bateau pour voir la localisation</div>"
)

map_output = widgets.Output()

# Chat (le handler sera défini plus loin en cellule 25)
chat_input = widgets.Text(
    placeholder='Posez une question sur les bateaux...',
    description='Question:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)
chat_output = widgets.HTML(
    value="<div style='padding:10px;background:#fff8dc;border-radius:5px;color:#2c3e50;font-weight:500;'>💬 Chatbot prêt! Posez vos questions sur les bateaux.</div>"
)
chat_button = widgets.Button(
    description='Envoyer',
    button_style='primary',
    icon='paper-plane'
)

def update_boat_info(change):
    """Met à jour météo + localisation + carte quand un MMSI est sélectionné"""
    global current_boat, current_weather, current_location
    global last_selected_mmsi, last_selected_name

    selected_mmsi = change['new']  # valeur du Dropdown = MMSI (int/str) ou None

    # ✅ IMPORTANT: Mettre à jour la référence globale
    last_selected_mmsi = str(selected_mmsi) if selected_mmsi else None

    # Rien sélectionné
    if not selected_mmsi:
        last_selected_name = None
        weather_output.value = "<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
        location_output.value = "<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                live_df = df_boats.copy()
            display(create_boat_map(live_df, selected_mmsi=None))
        return

    # Normaliser le MMSI sélectionné
    selected_mmsi_str = str(selected_mmsi)
    last_selected_mmsi = selected_mmsi_str

    # Chercher le navire dans le flux AIS
    v = vessels_state.get(selected_mmsi) or vessels_state.get(int(selected_mmsi)) or vessels_state.get(str(selected_mmsi))
    name = (v.get("name") or "").strip() if v else ""
    last_selected_name = name or f"MMSI {selected_mmsi_str}"

    # Coords
    lat = v.get("lat") if v else None
    lon = v.get("lon") if v else None

    # Fallback: si pas de coords côté AIS, tenter dans df_boats par nom
    if (lat is None or lon is None) and name:
        match = df_boats[df_boats['boat_name'] == name].head(1)
        if not match.empty:
            lat, lon = float(match.iloc[0]['latitude']), float(match.iloc[0]['longitude'])

    # Météo + Localisation si coords OK
    if lat is not None and lon is not None:
        weather_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement météo...</div>"
        wdata = get_weather_data(lat, lon, OPENWEATHER_API_KEY)
        current_weather = wdata
        weather_info = format_weather_info(wdata)
        weather_output.value = f"<div style='padding:10px;background:#f0f8ff;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌤️ Météo pour {last_selected_name}</h3><div style='font-weight:500;'>{weather_info}</div></div>"

        location_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement localisation...</div>"
        ldata = get_location_info(lat, lon)
        current_location = ldata
        loc_info = format_location_info(ldata)
        location_output.value = f"<div style='padding:10px;background:#f0fff0;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌍 Localisation de {last_selected_name}</h3><div style='font-weight:500;'>{loc_info}</div></div>"
    else:
        weather_output.value = "<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire pour le moment.</div>"
        location_output.value = "<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire pour le moment.</div>"

    # ✅ CORRECTION PRINCIPALE: Redessiner la carte avec sélection en ROUGE
    with map_output:
        clear_output(wait=True)
        try:
            live_df = vessels_state_to_df()
            # ✅ Passer le MMSI sélectionné pour mettre l'icône en rouge
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        except Exception:
            live_df = df_boats.copy()
            # ✅ Fallback: créer une colonne mmsi temporaire si elle n'existe pas
            if 'mmsi' not in live_df.columns:
                live_df['mmsi'] = range(1000000, 1000000 + len(live_df))
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))

# Connecter le sélecteur (⚠️ le chat sera branché en cellule 25)
boat_selector.observe(update_boat_info, names='value')

print("✅ Interface (widgets) initialisée – la liste sera alimentée par la cellule de refresh.")



✅ Interface (widgets) initialisée – la liste sera alimentée par la cellule de refresh.


# 📜 Widgets historique discussion (output + bouton effacement)

In [10]:
history_output = widgets.HTML(
    value="<div style='padding:10px;background:#f7f8fa;border-radius:6px;color:#2c3e50;'>Aucun échange pour l'instant.</div>"
)

clear_history_button = widgets.Button(
    description='🗑️ Effacer l\'historique',
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='200px')
)

print("✅ Widgets d'historique créés!")

✅ Widgets d'historique créés!


# 💻 Widgets interface + handlers (sélection, météo, chat, événements)

In [11]:
# Variables globales pour l'état de l'application
current_boat = None
current_weather = None
current_location = None

# ✅ Variables globales pour synchronisation avec l'agent IA
last_selected_mmsi = None
last_selected_name = None

# Widgets de l'interface
boat_selector = widgets.Dropdown(
    options=[('Sélectionnez un bateau', None)],  # ✅ Sera rempli dynamiquement par refresh_ui_from_stream()
    value=None,
    description='Bateau:',
    style={'description_width': 'initial'}
)

weather_output = widgets.HTML(
    value="<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
)

location_output = widgets.HTML(
    value="<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
)

map_output = widgets.Output()

# Chatbot
chat_input = widgets.Text(
    placeholder='Posez une question sur les bateaux...',
    description='Question:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

chat_output = widgets.HTML(
    value="<div style='padding: 10px; background: #fff8dc; border-radius: 5px; color: #2c3e50; font-weight: 500;'>💬 Chatbot prêt! Posez vos questions sur les bateaux.</div>"
)

chat_button = widgets.Button(
    description='Envoyer',
    button_style='primary',
    icon='paper-plane'
)

def update_boat_info(change):
    """Met à jour météo + localisation + carte quand un MMSI est sélectionné"""
    global current_boat, current_weather, current_location
    global last_selected_mmsi, last_selected_name

    selected_mmsi = change['new']  # valeur du Dropdown = MMSI (int/str) ou None

    # ✅ IMPORTANT: Mettre à jour la référence globale
    last_selected_mmsi = str(selected_mmsi) if selected_mmsi else None

    # Rien sélectionné
    if not selected_mmsi:
        last_selected_name = None
        weather_output.value = "<div style='padding: 10px; background: #f0f8ff; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la météo</div>"
        location_output.value = "<div style='padding: 10px; background: #f0fff0; border-radius: 5px; color: #2c3e50; font-weight: 500;'>Sélectionnez un bateau pour voir la localisation</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                # Fallback: carte vide si pas de données AIS
                live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=None))
        return

    # Normaliser le MMSI sélectionné
    selected_mmsi_str = str(selected_mmsi)
    last_selected_mmsi = selected_mmsi_str

    # ✅ Chercher le navire dans le flux AIS temps réel
    v = vessels_state.get(selected_mmsi) or vessels_state.get(int(selected_mmsi)) or vessels_state.get(str(selected_mmsi))
    name = (v.get("name") or "").strip() if v else ""
    last_selected_name = name or f"MMSI {selected_mmsi_str}"

    # Coords depuis AIS
    lat = v.get("lat") if v else None
    lon = v.get("lon") if v else None

    # ✅ Pas de fallback sur df_boats (données fictives) - on utilise que les données AIS réelles
    if lat is None or lon is None:
        weather_output.value = "<div style='padding:10px;background:#f0f8ff;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire AIS pour le moment.</div>"
        location_output.value = "<div style='padding:10px;background:#f0fff0;border-radius:5px;color:#2c3e50;font-weight:500;'>Coordonnées indisponibles pour ce navire AIS pour le moment.</div>"
        with map_output:
            clear_output(wait=True)
            try:
                live_df = vessels_state_to_df()
            except Exception:
                live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        return

    display_name = name or f"MMSI {selected_mmsi_str}"

    # Météo (OpenWeather)
    weather_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement météo...</div>"
    w = get_weather_data(lat, lon, OPENWEATHER_API_KEY)
    current_weather = w
    weather_info = format_weather_info(w)
    weather_output.value = f"<div style='padding:10px;background:#f0f8ff;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌤️ Météo pour {display_name}</h3><div style='font-weight:500;'>{weather_info}</div></div>"

    # Localisation (reverse geocoding)
    location_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Chargement localisation...</div>"
    ldata = get_location_info(lat, lon)
    current_location = ldata
    loc_info = format_location_info(ldata)
    location_output.value = f"<div style='padding:10px;background:#f0fff0;border-radius:5px;white-space: pre-line;color:#2c3e50;'><h3 style='color:#34495e;margin-top:0;'>🌍 Localisation de {display_name}</h3><div style='font-weight:500;'>{loc_info}</div></div>"

    # ✅ Redessiner la carte avec sélection en ROUGE
    with map_output:
        clear_output(wait=True)
        try:
            live_df = vessels_state_to_df()
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))
        except Exception:
            live_df = pd.DataFrame(columns=["mmsi","boat_name","latitude","longitude","boat_type","flag_country"])
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi_str))

# ✅ Connexion des événements (le chat sera géré par la cellule handler chat avancé)
boat_selector.observe(update_boat_info, names='value')

print("✅ Interface widgets configurée (sera alimentée par le flux AIS)!")


✅ Interface widgets configurée (sera alimentée par le flux AIS)!


# 🌊 Configuration zone Méditerranée + structure données navires AIS

In [12]:
import asyncio, json, time
import nest_asyncio, websockets
import pandas as pd

nest_asyncio.apply()

# Bounding box Méditerranée (approx, utile pour filtrer côté client)
MED_MIN_LAT, MED_MAX_LAT = 30.0681, 47.3764
MED_MIN_LON, MED_MAX_LON = -6.0326, 42.3550

def in_med(lat, lon):
    return (MED_MIN_LAT <= lat <= MED_MAX_LAT) and (MED_MIN_LON <= lon <= MED_MAX_LON)

# État courant des navires vus (clé = MMSI)
# On stocke ce qu'on sait : nom, lat, lon, sog, cog, timestamp, type, pavillon (si dispo)
vessels_state = {}

def vessels_state_to_df():
    rows = []
    for mmsi, v in vessels_state.items():
        lat = v.get("lat"); lon = v.get("lon")
        if lat is None or lon is None:
            continue
        rows.append({
            "mmsi": int(mmsi) if isinstance(mmsi, (int, str)) and str(mmsi).isdigit() else str(mmsi),
            "boat_name": (v.get("name") or "").strip() or f"MMSI {mmsi}",
            "latitude": float(lat),
            "longitude": float(lon),
            "boat_type": v.get("type") or "Unknown",
            "flag_country": v.get("flag") or "Inconnu",
        })
    cols = ["mmsi","boat_name","latitude","longitude","boat_type","flag_country"]
    return pd.DataFrame(rows, columns=cols)



# 🔍 Helper résolution coordonnées navire sélectionné (nom/MMSI → lat/lon)

In [13]:
# 16 - 🔎 Helper: retrouver lat/lon d'un bateau sélectionné depuis le flux AIS

def resolve_selection_to_coords(selection_label):
    """
    Retrouve (lat, lon, name) d'un bateau sélectionné en se basant sur vessels_state (AIS réel).
    - selection_label est la valeur du Dropdown (actuellement le nom ou "MMSI NNN...")
    - retourne (lat, lon, name) ou (None, None, None) si non trouvé
    """
    if not selection_label:
        return None, None, None

    # 1) correspondance exacte sur le nom reçu dans ShipStaticData
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if name and name == selection_label:
            return v.get("lat"), v.get("lon"), name

    # 2) correspondance sur libellé "MMSI 123456789" si présent dans la liste
    if selection_label.startswith("MMSI "):
        try:
            mmsi_int = int(selection_label.split("MMSI ", 1)[1])
            v = vessels_state.get(mmsi_int) or vessels_state.get(str(mmsi_int))
            if v and v.get("lat") is not None and v.get("lon") is not None:
                return v.get("lat"), v.get("lon"), v.get("name") or selection_label
        except Exception:
            pass

    # 3) pas trouvé côté AIS
    return None, None, None


# 🛰️ Client WebSocket AISStream avec gestion d'erreurs robuste

In [14]:
async def aisstream_connect_and_consume():
    """
    Connexion au WebSocket AISStream avec gestion d'erreurs robuste
    """
    uri = "wss://stream.aisstream.io/v0/stream"

    try:
        async with websockets.connect(
            uri,
            ping_interval=20,
            ping_timeout=20,
            close_timeout=10,
            max_size=2**20  # 1MB max message size
        ) as ws:

            # Message de souscription
            subscribe_msg = {
                "APIKey": AISSTREAM_API_KEY,
                "BoundingBoxes": [[[MED_MIN_LAT, MED_MIN_LON], [MED_MAX_LAT, MED_MAX_LON]]],
                "FilterMessageTypes": ["PositionReport", "ShipStaticData"]
            }

            await ws.send(json.dumps(subscribe_msg))
            print("✅ Souscription AISStream envoyée")

            message_count = 0
            async for raw in ws:
                try:
                    msg = json.loads(raw)
                    message_count += 1

                    # Log périodique
                    if message_count % 100 == 0:
                        print(f"📊 {message_count} messages AIS reçus, {len(vessels_state)} navires en mémoire")

                    mtype = msg.get("MessageType")
                    data = msg.get("Message", {})

                    # Position des navires
                    if mtype == "PositionReport":
                        pr = data.get("PositionReport", {})
                        mmsi = pr.get("UserID")
                        lat = pr.get("Latitude")
                        lon = pr.get("Longitude")
                        sog = pr.get("Sog")
                        cog = pr.get("Cog")

                        if mmsi and lat is not None and lon is not None and in_med(lat, lon):
                            v = vessels_state.setdefault(mmsi, {})
                            v.update({
                                "lat": lat, "lon": lon, "sog": sog, "cog": cog,
                                "last_ts": time.time()
                            })

                    # Données statiques
                    elif mtype == "ShipStaticData":
                        sd = data.get("ShipStaticData", {})
                        mmsi = sd.get("UserID")
                        name = (sd.get("Name") or "").strip()
                        stype = sd.get("Type")

                        if mmsi:
                            v = vessels_state.setdefault(mmsi, {})
                            if name:
                                v["name"] = name
                            if stype:
                                v["type"] = str(stype)

                except json.JSONDecodeError:
                    continue  # Ignorer les messages malformés
                except Exception as e:
                    print(f"⚠️ Erreur traitement message AIS: {e}")
                    continue

    except websockets.exceptions.ConnectionClosedError as e:
        print(f"🔌 Connexion WebSocket fermée: {e}")
        raise
    except Exception as e:
        print(f"❌ Erreur connexion AISStream: {e}")
        raise

print("✅ Client WebSocket AISStream configuré avec gestion d'erreurs")



✅ Client WebSocket AISStream configuré avec gestion d'erreurs


# ▶️ Démarrage flux AISStream avec reconnexion automatique et gestion d'erreurs

In [15]:
import asyncio
import time

# Variable globale pour contrôler la tâche
stream_task = None

async def aisstream_connect_with_retry():
    """
    Lance le flux AISStream avec gestion d'erreurs et reconnexion automatique
    """
    retry_count = 0
    max_retries = 5
    base_delay = 5  # secondes

    while retry_count < max_retries:
        try:
            print(f"🔄 Tentative de connexion AISStream #{retry_count + 1}")
            await aisstream_connect_and_consume()
            # Si on arrive ici, la connexion s'est fermée normalement
            print("⚠️ Connexion AISStream fermée normalement")
            break

        except Exception as e:
            retry_count += 1
            error_msg = str(e)

            if "ConnectionClosedError" in error_msg:
                print(f"⚠️ Connexion AISStream fermée (tentative {retry_count}/{max_retries})")
            elif "TimeoutError" in error_msg:
                print(f"⏰ Timeout AISStream (tentative {retry_count}/{max_retries})")
            else:
                print(f"❌ Erreur AISStream: {error_msg} (tentative {retry_count}/{max_retries})")

            if retry_count < max_retries:
                delay = base_delay * (2 ** (retry_count - 1))  # Backoff exponentiel
                print(f"⏳ Reconnexion dans {delay}s...")
                await asyncio.sleep(delay)
            else:
                print("❌ Nombre maximum de tentatives atteint. Arrêt du flux AIS.")
                break

def start_ais_stream():
    """
    Démarre le flux AISStream de manière sécurisée
    """
    global stream_task

    # Arrêter la tâche précédente si elle existe
    if stream_task and not stream_task.done():
        stream_task.cancel()
        print("🛑 Arrêt de la tâche AIS précédente")

    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    # Créer la nouvelle tâche avec gestion d'erreurs
    stream_task = loop.create_task(aisstream_connect_with_retry())

    # Ajouter un callback pour gérer les exceptions non récupérées
    def handle_task_exception(task):
        if task.exception():
            print(f"⚠️ Tâche AIS terminée avec exception: {task.exception()}")
        else:
            print("✅ Tâche AIS terminée normalement")

    stream_task.add_done_callback(handle_task_exception)
    print("🚀 Flux AISStream lancé avec gestion d'erreurs (zone Méditerranée)")

# Démarrer le flux
start_ais_stream()


🚀 Flux AISStream lancé avec gestion d'erreurs (zone Méditerranée)


# 🔄 Fonctions refresh interface (dropdown + carte) depuis flux AIS

In [16]:
from IPython.display import clear_output, display

def refresh_ui_from_stream(selected_mmsi=None, max_ships=300):
    live_df = vessels_state_to_df()

    # Même si vide : on affiche la carte (avec EMODnet) et un message
    if live_df.empty:
        with map_output:
            clear_output(wait=True)
            display(create_boat_map(live_df, selected_mmsi=selected_mmsi))
        print("⏳ En attente de premiers messages AIS…")
        return

    if len(live_df) > max_ships:
        live_df = live_df.head(max_ships)

    # Options du Dropdown: étiquette lisible, valeur = MMSI (unique)
    def labelize(row):
        n = (row['boat_name'] or '').strip()
        mm = row['mmsi']
        return f"{n} — MMSI {mm}" if n else f"MMSI {mm}"

    names = [ (labelize(r), r['mmsi']) for _, r in live_df.sort_values('boat_name').iterrows() ]
    new_options = [('Sélectionnez un bateau', None)] + names

    try:
        boat_selector.options = new_options
        if selected_mmsi is not None and any(v == selected_mmsi for _, v in new_options):
            boat_selector.value = selected_mmsi
        else:
            boat_selector.value = None
    except Exception:
        pass

    with map_output:
        clear_output(wait=True)
        display(create_boat_map(live_df, selected_mmsi=selected_mmsi))


    # (Option : mettre à jour ton chatbot avec ce DF en contexte si tu veux)


def run_refresh_loop(seconds=10, iterations=30):
    """
    Rafraîchir automatiquement toutes les X secondes pendant N itérations.
    """
    sel = None
    for i in range(iterations):
        print(f"🔄 Refresh {i+1}/{iterations}")
        refresh_ui_from_stream(selected_boat_name=sel)
        time.sleep(seconds)
        clear_output(wait=True)
    print("✅ Boucle de refresh terminée.")


# ⏰ Premier refresh interface + refresh manuel (à relancer pour actualiser)

In [17]:
# Attendre que le flux AIS commence à recevoir des données
time.sleep(5)

# Premier refresh pour afficher les navires détectés
refresh_ui_from_stream()

print("✅ Premier refresh effectué")
print("📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule")
print("   ou utilisez la boucle automatique ci-dessous")

# OU lancer une boucle auto (toutes les 10s, 30 fois) :
# run_refresh_loop(seconds=10, iterations=30)

print("\n🔄 Options de refresh :")
print("• Relancer cette cellule manuellement quand vous voulez actualiser")
print("• Décommenter la ligne run_refresh_loop() pour un refresh automatique")
print("• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh")


⏳ En attente de premiers messages AIS…
✅ Premier refresh effectué
📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule
   ou utilisez la boucle automatique ci-dessous

🔄 Options de refresh :
• Relancer cette cellule manuellement quand vous voulez actualiser
• Décommenter la ligne run_refresh_loop() pour un refresh automatique
• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh


# 🧰 Fonctions utilitaires (Haversine, recherche navires, météo, bathymétrie, Beaufort)

In [18]:
import math, requests, difflib, unicodedata, datetime as dt

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2 * R * math.asin(math.sqrt(a))

def _norm(s: str) -> str:
    if not s: return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.lower().strip()

def vessel_candidates(query: str, k: int = 10):
    qn = _norm(query or "")
    by_name = []
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if not name: continue
        nn = _norm(name)
        sub = (qn in nn) if qn else False
        ratio = difflib.SequenceMatcher(None, qn, nn).ratio() if qn else 0
        score = 1.0 if sub else ratio
        by_name.append((score, str(mmsi), name))
    by_name.sort(reverse=True, key=lambda x: x[0])
    return [(m, n, s) for s, m, n in by_name[:k]]

def resolve_vessel_query(query: str, prefer_mmsi: str = None):
    """Essaie d'abord le MMSI sélectionné, sinon la requête (MMSI ou nom approx)."""
    # 1) MMSI favori
    if prefer_mmsi:
        v = vessels_state.get(int(prefer_mmsi)) or vessels_state.get(str(prefer_mmsi))
        if v and v.get("lat") is not None and v.get("lon") is not None:
            name = (v.get("name") or "").strip() or f"MMSI {prefer_mmsi}"
            return str(prefer_mmsi), v, name

    # 2) MMSI repéré dans la requête
    import re
    m = re.search(r'(\d{7,9})', str(query or ""))
    if m:
        mm = m.group(1)
        v = vessels_state.get(int(mm)) or vessels_state.get(mm)
        if v and v.get("lat") is not None and v.get("lon") is not None:
            name = (v.get("name") or "").strip() or f"MMSI {mm}"
            return str(mm), v, name

    # 3) nom approx
    qn = _norm(query or "")
    idx = {}
    for mmsi, v in vessels_state.items():
        name = (v.get("name") or "").strip()
        if name: idx[_norm(name)] = (str(mmsi), v, name)
    if qn in idx: return idx[qn]
    for nn, triple in idx.items():
        if qn and qn in nn: return triple
    cands = vessel_candidates(query, k=1)
    if cands:
        mmsi, name, _s = cands[0]
        v = vessels_state.get(int(mmsi)) or vessels_state.get(str(mmsi))
        if v: return str(mmsi), v, name

    return None, None, None

# --- OpenWeather current (lat/lon)
def openweather_current(lat: float, lon: float, api_key: str):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"lat": lat, "lon": lon, "appid": api_key, "units": "metric", "lang": "fr"}
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    return r.json()

# --- EMODnet Bathymetry REST: depth at point (WKT POINT(lon lat))
def emodnet_depth_sample(lat: float, lon: float):
    wkt = f"POINT({lon} {lat})"
    url = "https://rest.emodnet-bathymetry.eu/depth_sample"
    r = requests.get(url, params={"geom": wkt}, timeout=10)
    r.raise_for_status()
    return r.json()  # ex: {"min":31.2,"max":31.3,"avg":31.25,"smoothed":30.95,...}

# --- Beaufort
def bft_from_ms(v_ms: float) -> int:
    if v_ms is None: return None
    bounds = [0.3,1.6,3.4,5.5,8.0,10.8,13.9,17.2,20.8,24.5,28.5,32.7]
    for i,t in enumerate(bounds):
        if v_ms < t: return i
    return 12

def bft_label(b: int) -> str:
    labels = {
        0:"Calme",1:"Très légère brise",2:"Légère brise",3:"Brise légère",
        4:"Petite brise",5:"Bonne brise",6:"Vent frais",7:"Grand frais",
        8:"Coup de vent",9:"Fort coup de vent",10:"Tempête",11:"Violente tempête",12:"Ouragan"
    }
    return labels.get(b, "")



# 🛠️ Tools LangChain (ais_lookup + risk_assess) pour agent IA

In [19]:
# 🛠️ Tools LangChain (ais_lookup + risk_assess) pour agent IA

from langchain_core.tools import tool

@tool
def ais_lookup(vessel_query: str, selected_mmsi_hint: str = ""):
    """
    Trouve un navire par NOM/MMSI.
    Essaie d'abord le MMSI sélectionné dans l'UI (selected_mmsi_hint), puis la requête utilisateur.
    Met à jour 'last_ref' si trouvé.
    """
    global last_ref
    hint = selected_mmsi_hint or (last_selected_mmsi or "")
    mmsi, v, name = resolve_vessel_query(vessel_query, prefer_mmsi=hint)
    if not v:
        cands = vessel_candidates(vessel_query, k=5)
        return {"status": "not_found", "candidates": [{"mmsi": m, "name": n} for (m, n, _s) in cands]}
    lat, lon = v.get("lat"), v.get("lon")
    last_ref = {"mmsi": str(mmsi), "name": name, "lat": lat, "lon": lon}
    return {
        "status": "ok",
        "mmsi": str(mmsi),
        "name": name,
        "lat": lat, "lon": lon,
        "sog_kn": v.get("sog"),
        "cog_deg": v.get("cog"),
        "draught_m": (0.1*float(v.get("draught"))) if (v.get("draught") not in (None, 0, 255)) else None
    }

@tool
def risk_assess(vessel_query: str = "", neighbor_radius_nm: float = 2.0):
    """
    Évalue le risque maritime pour un navire donné.
    Analyse: conditions météo, profondeur, houle, trafic local, tirant d'eau si connu.
    Met à jour 'last_ref' si trouvé.
    """
    global last_ref
    mmsi, v, name = resolve_vessel_query(vessel_query, prefer_mmsi=last_selected_mmsi)
    if not v:
        return {"status": "not_found", "candidates": [{"mmsi": m, "name": n} for (m, n, _s) in vessel_candidates(vessel_query, k=5)]}

    lat, lon = v.get("lat"), v.get("lon")
    if lat is None or lon is None:
        return {"status": "no_coords", "message": "Coordonnées indisponibles pour ce navire"}

    last_ref = {"mmsi": str(mmsi), "name": name, "lat": lat, "lon": lon}

    try:
        # --- Météo avec gestion d'erreur ---
        wind, bft = None, None
        try:
            w = openweather_current(lat, lon, OPENWEATHER_API_KEY)
            wind = w.get("wind", {}).get("speed")  # m/s
            bft = bft_from_ms(wind) if wind is not None else None
        except Exception as e:
            print(f"Erreur météo: {e}")

        # --- Bathymétrie avec gestion d'erreur ---
        depth_m = None
        try:
            bathy = emodnet_depth_sample(lat, lon)
            raw = bathy.get("avg") or bathy.get("smoothed")
            depth_m = abs(float(raw)) if raw is not None else None
        except Exception as e:
            print(f"Erreur bathymétrie: {e}")

        # --- Houle (simulation simple) ---
        swh = None
        # Note: Pour la houle, vous pourriez intégrer Copernicus Marine plus tard

        # --- Analyse du trafic local ---
        neighbors_details = []
        for mm, vv in vessels_state.items():
            if str(mm) == str(mmsi):
                continue
            la, lo = vv.get("lat"), vv.get("lon")
            if la is None or lo is None:
                continue

            try:
                rng_m = haversine_m(lat, lon, la, lo)
                if rng_m <= neighbor_radius_nm * 1852.0:
                    neighbors_details.append({
                        "mmsi": str(mm),
                        "name": (vv.get("name") or "").strip() or f"MMSI {mm}",
                        "range_nm": round(rng_m / 1852.0, 2),
                        "sog": vv.get("sog"),
                        "cog": vv.get("cog")
                    })
            except Exception:
                continue

        neighbors_details.sort(key=lambda x: x["range_nm"])
        neighbors = len(neighbors_details)
        neighbors_top5 = neighbors_details[:5]

        # --- Calcul du tirant d'eau et UKC ---
        draught_m = None
        try:
            dr = v.get("draught")
            if dr not in (None, 0, 255):
                draught_m = 0.1 * float(dr)
        except Exception:
            pass

        ukc = None
        if depth_m is not None and draught_m is not None:
            ukc = depth_m - draught_m - 0.5  # Marge de sécurité de 0.5m

        # --- Calcul du score de risque ---
        score, notes = 0, []

        # Risque météorologique
        if bft is not None:
            if bft >= 8:
                score += 3
                notes.append(f"Vent très fort Beaufort {bft} - Conditions dangereuses")
            elif bft >= 6:
                score += 2
                notes.append(f"Vent fort Beaufort {bft} - Navigation difficile")
            elif bft >= 5:
                score += 1
                notes.append(f"Vent modéré Beaufort {bft} - Vigilance requise")

        # Risque de profondeur/échouage
        if ukc is not None:
            if ukc < 0.5:
                score += 4
                notes.append(f"UKC critique {ukc:.1f}m - Risque d'échouage imminent")
            elif ukc < 1.0:
                score += 3
                notes.append(f"UKC très faible {ukc:.1f}m - Risque d'échouage élevé")
            elif ukc < 2.0:
                score += 2
                notes.append(f"UKC faible {ukc:.1f}m - Navigation délicate")
            elif ukc < 3.0:
                score += 1
                notes.append(f"UKC modérée {ukc:.1f}m - Prudence recommandée")
        elif depth_m is not None and draught_m is None:
            if depth_m < 3:
                score += 3
                notes.append(f"Très faible fond {depth_m:.1f}m - Tirant inconnu, risque d'échouage")
            elif depth_m < 10:
                score += 2
                notes.append(f"Faible fond {depth_m:.1f}m - Tirant inconnu, prudence requise")
            elif depth_m < 15:
                score += 1
                notes.append(f"Fond modéré {depth_m:.1f}m - Tirant inconnu, surveillance recommandée")

        # Risque de trafic
        if neighbors >= 15:
            score += 2
            notes.append(f"Trafic très dense ({neighbors} navires ≤{neighbor_radius_nm}nm) - Risque de collision")
        elif neighbors >= 10:
            score += 2
            notes.append(f"Trafic dense ({neighbors} navires ≤{neighbor_radius_nm}nm) - Surveillance radar renforcée")
        elif neighbors >= 5:
            score += 1
            notes.append(f"Trafic modéré ({neighbors} navires ≤{neighbor_radius_nm}nm) - Veille normale")

        # Détermination du niveau de risque
        if score >= 6:
            level = "critique"
        elif score >= 4:
            level = "élevé"
        elif score >= 2:
            level = "modéré"
        else:
            level = "faible"

        return {
            "status": "ok",
            "mmsi": str(mmsi),
            "name": name,
            "lat": lat,
            "lon": lon,
            "wind_mps": wind,
            "beaufort": bft,
            "depth_m_LAT": depth_m,
            "swh_m": swh,
            "neighbors": neighbors,
            "neighbors_top5": neighbors_top5,
            "draught_m": draught_m,
            "ukc_m": ukc,
            "risk_score": score,
            "risk_level": level,
            "notes": notes
        }

    except Exception as e:
        return {
            "status": "error",
            "message": f"Erreur lors de l'évaluation de risque: {str(e)}",
            "mmsi": str(mmsi) if mmsi else None,
            "name": name if name else None
        }

# ✅ IMPORTANT: Rassembler tous les tools dans une liste
tools = [ais_lookup, risk_assess]

print(f"✅ {len(tools)} tools LangChain définis: {[t.name for t in tools]}")


✅ 2 tools LangChain définis: ['ais_lookup', 'risk_assess']


# 🤖 Agent LangChain avec mémoire conversationnelle et fallback multi-modèles

In [20]:
# 🤖 Agent LangChain avec mémoire conversationnelle et fallback multi-modèles

import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.messages import HumanMessage, AIMessage

PRIMARY_MODEL   = "deepseek/deepseek-chat-v3.1:free"
FALLBACK_MODELS = ["openai/gpt-4o-mini"]  # fallback tool-calling

def _make_llm(model_name: str) -> ChatOpenAI:
    # OpenRouter est compatible OpenAI : on pointe ChatOpenAI vers OpenRouter
    return ChatOpenAI(
        model=model_name,
        temperature=0.2,
        api_key=OPENROUTER_API_KEY,
        base_url="https://openrouter.ai/api/v1",
        default_headers={
            "HTTP-Referer": "https://colab.research.google.com",
            "X-Title": "Boat Tracker Agent"
        },
    )

# 🧠 mémoire locale (liste de messages LangChain) + fenêtre glissante
chat_memory_lc = []          # HumanMessage / AIMessage
MEM_WINDOW_TURNS = 12        # on garde ~12 tours (≈ 24 messages)

def _prune_memory():
    excess = len(chat_memory_lc) - (2 * MEM_WINDOW_TURNS)
    if excess > 0:
        del chat_memory_lc[:excess]

def get_chat_history():
    return chat_memory_lc

def append_chat_history(user_text=None, ai_text=None):
    if user_text is not None:
        chat_memory_lc.append(HumanMessage(content=user_text))
    if ai_text is not None:
        chat_memory_lc.append(AIMessage(content=ai_text))
    _prune_memory()

# 👉 Référence implicite au "dernier navire utilisé" (mise à jour par les tools)
if "last_ref" not in globals():
    last_ref = {}  # ex: {"mmsi":"123456789","name":"MAYAR","lat":..., "lon":...}

system_txt = """Tu es un assistant naval opérationnel expert, spécialisé dans l'aide à la navigation maritime.

CONTEXTE OPÉRATIONNEL:
- Tu assistes des marins professionnels dans leurs opérations
- Tes réponses doivent être claires, précises et adaptées au vocabulaire maritime
- Utilise toujours les unités nautiques (nœuds, milles nautiques, etc.)

RÈGLES DE RÉFÉRENCE:
- Si l'utilisateur mentionne un nom de navire, mémorise-le comme référence active
- Pour "ce navire", "il", "le navire" : utilise d'abord le navire mentionné dans la conversation courante
- En cas de doute, demande une clarification plutôt que de deviner

OUTILS OBLIGATOIRES:
- Pour toute question de position/localisation : utilise ais_lookup
- Pour l'évaluation de risque : utilise risk_assess

STYLE DE COMMUNICATION:
- Sois direct et factuel
- Structure tes réponses avec des puces claires
- Utilise des emojis pour la lisibilité (📍 🌊 ⚡ 🧭)
- Indique toujours le MMSI pour identification
- Signale les points d'attention avec ⚠️ ou 🚨

GESTION D'ERREURS:
- Si un outil échoue, explique clairement le problème
- Propose des alternatives quand possible
- Ne jamais inventer de données
"""

# 👇 Le prompt DOIT contenir 'agent_scratchpad' (MessagesPlaceholder) et on lui passe 'chat_history'
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_txt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

def _make_agent(model_name: str) -> AgentExecutor:
    llm = _make_llm(model_name)
    ag = create_tool_calling_agent(llm, tools, prompt)   # tool-calling agent moderne
    ex = AgentExecutor(agent=ag, tools=tools, verbose=False)
    ex._model_name = model_name
    return ex

agent = _make_agent(PRIMARY_MODEL)

def ask_naval_assistant(question: str):
    """Applique la mémoire + hint navire par défaut (UI/last_ref) et gère les fallbacks."""
    global agent
    history = get_chat_history()

    # Indice implicite pour le LLM : quel navire prendre "par défaut"
    last_hint = last_selected_mmsi or (last_ref.get("mmsi") if isinstance(last_ref, dict) else None) or "ND"
    enriched = f"{question}\n\n[Contexte UI/last_ref: MMSI par défaut={last_hint}]"

    candidates = [PRIMARY_MODEL] + FALLBACK_MODELS
    for model in candidates:
        if getattr(agent, "_model_name", None) != model:
            agent = _make_agent(model)
        for attempt in range(2):
            try:
                return agent.invoke({"input": enriched, "chat_history": history})
            except Exception as e:
                msg = str(e)
                # gestion simple du rate limit/provider
                if ("429" in msg) or ("rate limit" in msg.lower()) or ("temporarily rate-limited" in msg.lower()):
                    time.sleep(1.0 + attempt * 1.5)
                    continue
                raise

    return {"output": "⚠️ Tous les modèles sont momentanément indisponibles. Réessaie plus tard."}

print("✅ Agent LangChain configuré avec outils maritimes")

✅ Agent LangChain configuré avec outils maritimes


# 💬 Handler chat avancé avec formatage maritime et gestion historique

In [21]:
import re
from html import escape

def _extract_candidate_name(q: str) -> str:
    m = re.search(r"(navire|bateau)\s+([A-Za-z0-9\-_/\. ]{2,})", q, flags=re.I)
    return m.group(2).strip() if m else q.strip()

def _format_risk(r: dict) -> str:
    if r.get("status") != "ok":
        if r.get("status") == "not_found":
            cands = r.get("candidates") or []
            if cands:
                cand_str = ", ".join([f"{c['name']} (MMSI {c['mmsi']})" for c in cands])
                return "Navire introuvable dans le snapshot AIS. Vouliez-vous dire: " + cand_str + " ?"
            return "Navire introuvable dans le snapshot AIS pour l’instant."
        if r.get("status") == "no_coords":
            return "Coordonnées du navire indisponibles pour le moment."
        return "Données de risque indisponibles pour le moment."

    bft = r.get("beaufort")
    bft_txt = f"Bft {bft} ({bft_label(bft)})" if bft is not None else "ND"
    parts = []
    parts.append(f"Risque **{r['risk_level']}** (score {r['risk_score']}).")
    parts.append(f"Vent {bft_txt}")
    if r.get("swh_m") is not None: parts.append(f"Houle ~{r['swh_m']:.1f} m (Copernicus)")
    if r.get("depth_m_LAT") is not None: parts.append(f"Profondeur ~{r['depth_m_LAT']:.1f} m LAT")
    if r.get("ukc_m") is not None: parts.append(f"UKC ~{r['ukc_m']:.1f} m")
    parts.append(f"Trafic local: {r['neighbors']} bateau(x) ≤2 nm")
    neigh = r.get("neighbors_top5") or []
    if neigh:
        top = ", ".join([f"{n['name']} ({n['range_nm']} nm)" for n in neigh[:3]])
        parts.append(f"Plus proches: {top}")
    head = " ".join(parts)
    tail = f"Navire: {r['name']} (MMSI {r['mmsi']})."
    return head + "\n" + tail

def _format_risk_marine_friendly(r: dict, vessel_name: str = None) -> str:
    """Formate l'évaluation de risque dans un langage adapté aux marins"""
    if r.get("status") != "ok":
        if r.get("status") == "not_found":
            cands = r.get("candidates") or []
            if cands:
                cand_str = ", ".join([f"{c['name']} (MMSI {c['mmsi']})" for c in cands[:3]])
                return f"❌ Navire '{vessel_name}' non localisé dans la zone AIS. Navires similaires: {cand_str}"
            return f"❌ Navire '{vessel_name}' non détecté dans la zone de couverture AIS actuelle."
        return "❌ Évaluation de risque temporairement indisponible."

    # Nom du navire pour clarifier la réponse
    ship_name = vessel_name or r.get('name', 'Navire inconnu')
    mmsi = r.get('mmsi', 'N/A')

    # Niveau de risque avec emoji
    risk_level = r.get('risk_level', 'indéterminé')
    risk_emoji = "🟢" if risk_level == "faible" else "🟡" if risk_level == "modéré" else "🔴"

    # Construction du rapport
    report = [f"**{ship_name}** (MMSI {mmsi}) - Risque {risk_emoji} **{risk_level.upper()}**"]

    # Conditions météorologiques
    bft = r.get("beaufort")
    if bft is not None:
        wind_desc = bft_label(bft)
        wind_status = "🌪️ ATTENTION" if bft >= 7 else "⚠️ VIGILANCE" if bft >= 5 else "✅ Acceptable"
        report.append(f"• **Vent**: Beaufort {bft} ({wind_desc}) - {wind_status}")

    # Profondeur et risque d'échouage
    depth = r.get("depth_m_LAT")
    ukc = r.get("ukc_m")
    if ukc is not None:
        ukc_status = "🚨 CRITIQUE" if ukc < 1.0 else "⚠️ SERRÉ" if ukc < 2.0 else "✅ Sûr"
        report.append(f"• **Under Keel Clearance**: {ukc:.1f}m - {ukc_status}")
    elif depth is not None:
        depth_status = "🚨 TRÈS FAIBLE" if depth < 5 else "⚠️ FAIBLE" if depth < 15 else "✅ Suffisante"
        report.append(f"• **Profondeur**: {depth:.1f}m LAT - {depth_status}")
        if r.get("draught_m") is None:
            report.append("  ℹ️ Tirant d'eau du navire inconnu - Prudence recommandée")

    # Houle
    swh = r.get("swh_m")
    if swh is not None:
        wave_status = "🌊 FORTE" if swh >= 3.0 else "⚠️ MODÉRÉE" if swh >= 1.5 else "✅ Calme"
        report.append(f"• **Houle**: {swh:.1f}m - {wave_status}")

    # Trafic maritime
    neighbors = r.get("neighbors", 0)
    traffic_status = "🚢 DENSE" if neighbors >= 10 else "⚠️ MODÉRÉ" if neighbors >= 5 else "✅ Fluide"
    report.append(f"• **Trafic local**: {neighbors} navire(s) dans 2nm - {traffic_status}")

    # Navires les plus proches
    neighbors_top = r.get("neighbors_top5", [])[:3]
    if neighbors_top:
        close_ships = ", ".join([f"{n['name']} ({n['range_nm']}nm)" for n in neighbors_top])
        report.append(f"• **Plus proches**: {close_ships}")

    # Recommandations
    notes = r.get("notes", [])
    if notes:
        report.append("**⚠️ Points d'attention:**")
        for note in notes:
            report.append(f"  - {note}")

    return "\n".join(report)

def _make_marine_friendly(text: str) -> str:
    """Adapte le langage technique pour les marins"""
    # Remplacements pour un langage plus maritime
    replacements = {
        "latitude": "latitude",
        "longitude": "longitude",
        "MMSI": "MMSI",
        "nœuds": "nœuds",
        "degrés": "°",
        "mètres": "m",
        "kilomètres": "km",
        "nautique": "nautique",
        "vitesse sur le fond": "vitesse",
        "cap sur le fond": "route",
        "à l'arrêt": "stoppé",
        "immobilisé": "au mouillage/stoppé"
    }

    result = text
    for old, new in replacements.items():
        result = result.replace(old, new)

    # Améliorer la structure des réponses
    if "Position:" in result:
        result = result.replace("Position:", "📍 **Position actuelle:**")
    if "Vitesse:" in result:
        result = result.replace("Vitesse:", "⚡ **Vitesse:**")
    if "Cap:" in result:
        result = result.replace("Cap:", "🧭 **Route:**")

    return result

def _render_history_html():
    if not chat_memory_lc:
        return """<div style='padding:15px;background:#f7f8fa;border-radius:8px;color:#2c3e50;text-align:center;font-style:italic;'>
                    🤖 Aucun échange pour l'instant. Posez votre première question !
                  </div>"""

    rows = []
    for i, msg in enumerate(chat_memory_lc):
        role = "Utilisateur" if isinstance(msg, type(HumanMessage(content=''))) else "Assistant"

        if role == "Utilisateur":
            cls = "background:linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%);border-left:4px solid #2196f3;"
            icon = "👤"
        else:
            cls = "background:linear-gradient(135deg, #f1f8e9 0%, #dcedc8 100%);border-left:4px solid #4caf50;"
            icon = "🤖"

        content = escape(msg.content).replace("\n", "<br>")
        timestamp = f"Message {i//2 + 1}" if role == "Utilisateur" else ""

        rows.append(f"""
        <div style='margin:8px 0;padding:12px 15px;border-radius:8px;{cls}box-shadow:0 2px 4px rgba(0,0,0,0.1);'>
            <div style='display:flex;align-items:center;margin-bottom:5px;'>
                <span style='margin-right:8px;font-size:16px;'>{icon}</span>
                <b style='color:#1976d2;'>{role}</b>
                {f'<span style="margin-left:auto;font-size:12px;color:#666;">{timestamp}</span>' if timestamp else ''}
            </div>
            <div style='color:#2c3e50;line-height:1.4;'>{content}</div>
        </div>
        """)

    return f"""<div style='max-height:350px;overflow-y:auto;padding:10px;background:#fafafa;border-radius:8px;'>
                 {"".join(rows)}
               </div>"""

def handle_chat(_button):
    question = chat_input.value.strip()
    if not question:
        return

    # 💾 ajoute la question dans la mémoire AVANT l'appel agent
    append_chat_history(user_text=question)

    chat_output.value = "<div style='padding:10px;background:#fffacd;border-radius:5px;color:#2c3e50;font-weight:500;'>⏳ Réflexion en cours...</div>"

    try:
        qlow = question.lower()
        # Fast-path "risque"
        if "risque" in qlow and ("navire" in qlow or "bateau" in qlow or "il" in qlow or "lui" in qlow or "ce" in qlow):
            name_guess = _extract_candidate_name(question)
            vague = any(w in qlow for w in ["ce navire", "navire actuel", "bateau actuel", "ce bateau", "il", "lui"])
            if vague and last_selected_mmsi:
                name_guess = f"MMSI {last_selected_mmsi}"
            elif vague and isinstance(last_ref, dict) and last_ref.get("mmsi"):
                name_guess = f"MMSI {last_ref['mmsi']}"
            r = risk_assess.invoke({"vessel_query": name_guess})
            answer = _format_risk(r)
        else:
            # Agent tool-calling AVEC chat_history
            res = ask_naval_assistant(question)
            answer = res.get("output") or str(res)

    except Exception as e:
        answer = f"❌ Agent/Tools indisponibles: {e}"

    # 💾 ajoute la réponse dans la mémoire APRÈS
    append_chat_history(ai_text=answer)

    # Rendu UI
    chat_output.value = f"""<div style='padding:10px;background:#fff8dc;border-radius:5px;color:#2c3e50;'>
    <h4 style='color:#34495e;margin-top:0;'>❓ Question: {escape(question)}</h4>
    <h4 style='color:#34495e;'>🤖 Assistant IA (outillé):</h4>
    <p style='white-space: pre-wrap; font-weight: 500; line-height: 1.5;'>{answer}</p>
    </div>"""
    history_output.value = _render_history_html()
    chat_input.value = ""

# (Re)lier sans doublons
try:
    chat_button._click_handlers.callbacks = []
except Exception:
    pass
chat_button.on_click(handle_chat)

def handle_enter(_):
    if chat_input.value.strip():
        handle_chat(None)
chat_input.on_submit(handle_enter)

# Bouton "effacer l'historique"
def _clear_history(_btn):
    chat_memory_lc.clear()
    history_output.value = _render_history_html()
try:
    clear_history_button._click_handlers.callbacks = []
except Exception:
    pass
clear_history_button.on_click(_clear_history)

# Affiche l'historique courant au chargement
history_output.value = _render_history_html()

print("✅ Chat branché (mémoire multi-tours + référence implicite de navire).")


✅ Chat branché (mémoire multi-tours + référence implicite de navire).


# 🚀 Interface principale complète avec historique et instructions d'utilisation


In [22]:
# Affichage initial de la carte
with map_output:
    display(create_boat_map(df_boats))

# Création de l'interface avec un design moderne incluant l'historique
header = widgets.HTML(
    value="""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 20px; border-radius: 10px; margin-bottom: 20px;'>
        <h1 style='color: white; text-align: center; margin: 0; font-family: Arial, sans-serif;'>
            🚢 Système de Tracking Maritime
        </h1>
        <p style='color: #f0f0f0; text-align: center; margin: 10px 0 0 0;'>
            Suivez vos bateaux en temps réel avec météo, géolocalisation et assistance IA
        </p>
    </div>
    """
)

# Layout principal avec historique
main_layout = widgets.VBox([
    header,

    # Sélecteur de bateau
    widgets.HTML("<h2>🎯 Sélection du bateau</h2>"),
    boat_selector,

    # Layout horizontal pour la carte et la sidebar
    widgets.HBox([
        # Carte (côté gauche)
        widgets.VBox([
            widgets.HTML("<h2>🗺️ Carte de tracking</h2>"),
            map_output
        ], layout=widgets.Layout(width='60%')),

        # Sidebar (côté droit)
        widgets.VBox([
            widgets.HTML("<h2>📊 Informations</h2>"),
            weather_output,
            widgets.HTML("<br>"),
            location_output
        ], layout=widgets.Layout(width='40%', padding='0 0 0 20px'))
    ]),

    # Section chatbot avec historique
    widgets.HTML("<br><h2>🤖 Assistant IA Maritime</h2>"),

    # Zone de chat actuelle
    widgets.VBox([
        widgets.HTML("<h3>💬 Nouvelle question</h3>"),
        widgets.HBox([chat_input, chat_button], layout=widgets.Layout(width='100%')),
        chat_output
    ], layout=widgets.Layout(margin='0 0 20px 0')),

    # Zone d'historique
    widgets.VBox([
        widgets.HBox([
            widgets.HTML("<h3>📜 Historique des discussions</h3>"),
            clear_history_button
        ], layout=widgets.Layout(justify_content='space-between', align_items='center')),
        widgets.HTML(
            value="<div style='max-height: 400px; overflow-y: auto; border: 1px solid #ddd; border-radius: 5px;'></div>",
            layout=widgets.Layout(width='100%')
        ),
        history_output
    ], layout=widgets.Layout(
        border='1px solid #e0e0e0',
        border_radius='8px',
        padding='15px',
        background_color='#fafafa'
    ))

], layout=widgets.Layout(padding='20px'))

# Affichage de l'application
display(main_layout)

print("\n🎉 Application de tracking maritime lancée avec succès!")
print("\n📝 Instructions d'utilisation:")
print("1. 🔑 Remplacez les clés API par vos vraies clés dans la section 'Configuration des clés API'")
print("2. 🎯 Sélectionnez un bateau dans le menu déroulant")
print("3. 🗺️ Visualisez sa position sur la carte (point rouge)")
print("4. 🌤️ Consultez les informations météo dans la sidebar")
print("5. 🌍 Vérifiez la géolocalisation (pays, ville, zone maritime)")
print("6. 💬 Posez des questions à l'assistant IA sur les bateaux")
print("7. 📜 Consultez l'historique de vos discussions")
print("8. 🗑️ Effacez l'historique quand nécessaire")
print("\n✨ Fonctionnalités:")
print("• 🚢 Plusieurs bateaux réels avec clé API AIstream")
print("• 🌤️ Données météo en temps réel (OpenWeather)")
print("• 🌍 Géolocalisation précise (pays, ville, zone maritime)")
print("• 🤖 Chatbot IA intégré (Langchain + OpenRouter)")
print("• 📜 Historique de discussion persistant")
print("• 🎨 Interface moderne et responsive")
print("• 🗺️ Carte interactive avec marqueurs colorés + bathymetrie")


🎉 Application de tracking maritime lancée avec succès!

📝 Instructions d'utilisation:
1. 🔑 Remplacez les clés API par vos vraies clés dans la section 'Configuration des clés API'
2. 🎯 Sélectionnez un bateau dans le menu déroulant
3. 🗺️ Visualisez sa position sur la carte (point rouge)
4. 🌤️ Consultez les informations météo dans la sidebar
5. 🌍 Vérifiez la géolocalisation (pays, ville, zone maritime)
6. 💬 Posez des questions à l'assistant IA sur les bateaux
7. 📜 Consultez l'historique de vos discussions
8. 🗑️ Effacez l'historique quand nécessaire

✨ Fonctionnalités:
• 🚢 Plusieurs bateaux réels avec clé API AIstream
• 🌤️ Données météo en temps réel (OpenWeather)
• 🌍 Géolocalisation précise (pays, ville, zone maritime)
• 🤖 Chatbot IA intégré (Langchain + OpenRouter)
• 📜 Historique de discussion persistant
• 🎨 Interface moderne et responsive
• 🗺️ Carte interactive avec marqueurs colorés + bathymetrie


# ⏰ Premier refresh interface + refresh manuel (à relancer pour actualiser)

In [25]:
# Attendre que le flux AIS commence à recevoir des données
time.sleep(5)

# Premier refresh pour afficher les navires détectés
refresh_ui_from_stream()

print("✅ Premier refresh effectué")
print("📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule")
print("   ou utilisez la boucle automatique ci-dessous")

# OU lancer une boucle auto (toutes les 10s, 30 fois) :
# run_refresh_loop(seconds=10, iterations=30)

print("\n🔄 Options de refresh :")
print("• Relancer cette cellule manuellement quand vous voulez actualiser")
print("• Décommenter la ligne run_refresh_loop() pour un refresh automatique")
print("• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh")


⏳ En attente de premiers messages AIS…
✅ Premier refresh effectué
📝 IMPORTANT: Pour actualiser la liste des navires, relancez cette cellule
   ou utilisez la boucle automatique ci-dessous

🔄 Options de refresh :
• Relancer cette cellule manuellement quand vous voulez actualiser
• Décommenter la ligne run_refresh_loop() pour un refresh automatique
• Les nouveaux navires AIS apparaîtront dans le dropdown après refresh
